In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from target_encoding import TargetEncoderClassifier, TargetEncoder
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import cross_val_score
import json
from functools import partial
from dsb2019.models.coeff import ThresholdClassifier

from dsb2019.models.tracking import track_experiment, track_submission_info
from dsb2019.data.validation import InstallationFold, cross_validate, quad_kappa
from dsb2019.visualization import session_browser
from dsb2019.data import DATA_DIR
from dsb2019.models import MODELS_DIR
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import hyperopt
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK
tqdm.pandas()
pd.options.display.max_rows=999

In [3]:
train = pd.read_csv(DATA_DIR / 'raw/train.csv')
test = pd.read_csv(DATA_DIR / 'raw/test.csv')
train_labels = pd.read_csv(DATA_DIR / 'raw/train_labels.csv')
submission = pd.read_csv(DATA_DIR / 'raw/sample_submission.csv')

In [4]:
train.event_id

0           27253bdc
1           27253bdc
2           77261ab5
3           b2dba42b
4           1bb5fbdb
              ...   
11341037    ab3136ba
11341038    27253bdc
11341039    27253bdc
11341040    27253bdc
11341041    27253bdc
Name: event_id, Length: 11341042, dtype: object

In [5]:
games = ['Scrub-A-Dub', 'All Star Sorting',
       'Air Show', 'Crystals Rule', 
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Pan Balance', 'Happy Camel',
       'Leaf Leader']
worlds = ['NONE', 'MAGMAPEAK', 'TREETOPCITY', 'CRYSTALCAVES']

def unwrap_event_data(df):
    unwrapped=pd.DataFrame(data=list(df.event_data.apply(json.loads).values))
    return pd.concat([unwrapped.reset_index(),df.reset_index()],axis=1)


def process_installations(train_labels, train, process_log):
    result = []
    train=train.sort_values("timestamp")
    installations = train.groupby("installation_id")
    for i, game_session, title, installation_id, accuracy_group in tqdm(train_labels[["game_session", "title", "installation_id", "accuracy_group"]].itertuples(), 
                                                              total=len(train_labels)):
        player_log = installations.get_group(installation_id).reset_index()
        log_length = player_log[(player_log.game_session==game_session) & (player_log.title==title)].index[0]
        player_log = player_log.iloc[:(log_length + 1)]
        player_log["accuracy_group"] = accuracy_group
        player_log["target_game_session"] = game_session
        features = process_log(player_log)
        features["installation_id"] = installation_id
        features["accuracy_group"] = accuracy_group
        result.append(features)
    return pd.DataFrame(data=result).fillna(-1)


def make_counters(df, column):
    return df.groupby(column)[column].count().to_dict()

    
def process_log(df):
    assessment_title=df.title.iloc[-1]   
    world=df.world.iloc[-1]

    history = df.iloc[:-1]
    history = history[history.type.isin(["Game", "Assessment"])].copy()

    def calculate_ratios(df):
        n_correct=df.correct_move.sum()
        n_incorrect=df.wrong_move.sum()
        ratio=n_correct/(n_correct+n_incorrect)
        return n_correct, n_incorrect, ratio
    
    def make_move_stats(df, title,n_lags=2):
        df=df.copy()
        if len(df):
            df = unwrap_event_data(df)
        if "correct" in df.columns:
            df["correct_move"] = df.correct == True
            df["wrong_move"] = df.correct == False
        else:
            df["correct_move"]=False
            df["wrong_move"]=False
        result = []
        result.extend(zip([f"n_correct_{title}", f"n_incorrect_{title}", f"global_ratio_{title}"], calculate_ratios(df)))

        if n_lags:
            last_sessions = df.game_session.unique()[-n_lags:]
            for i in range(n_lags):
                if i < len(last_sessions): 
                    result.extend(zip([f"n_correct_{title}_{i}", f"n_incorrect_{title} {i}",f"ratio_{title}_{i}"], 
                                      calculate_ratios(df[df.game_session==last_sessions[i]])))
                else:
                    result.extend(zip([f"n_correct_{title}_{i}", f"n_incorrect_{title}_{i}",f"ratio_{title}_{i}"], [None, None, None]))
        return {k: v for k, v in result}
    
    
    result = {"title": games.index(assessment_title),
              "world": worlds.index(world),
              "n_activities": df[df.type=="Activity"].game_session.nunique(),
              "n_games": df[df.type=="Game"].game_session.nunique(),
              "event_code_count": df.event_code.nunique(),
              "event_id_count": df.event_id.nunique(),
              "title_count": df.title.nunique(),
              "session_id_count": df.game_session.nunique(),
              "n_actions": len(df),
              "world_title_count": df[df.world==world].title.nunique(),
             }
    for game in games:
        stats=history[history.title==game]
        stats_features=make_move_stats(stats, game)
        stats_features[f"{game}_event_code_count"] = stats.event_code.nunique()
        stats_features[f"{game}_event_id_count"] = stats.event_id.nunique()
        stats_features[f"{game}_session_id_count"] = stats.game_session.nunique()
        stats_features[f"{game}_n_actions"] = len(stats)
        result.update(stats_features)
        result.update({f"{game}_{k}": v for k, v in make_counters(stats, "event_id").items()})
        result.update({f"{game}_{k}": v for k, v in make_counters(stats, "event_code").items()})
    world_games = history[history.world==world]
    for game in games:
        stats=world_games[world_games.title==game]
        stats_features=make_move_stats(stats, game)
        stats_features = {f"world_{k}": v for k, v in stats_features.items()}
        stats_features[f"world_{game}_event_code_count"] = stats.event_code.nunique()
        stats_features[f"world_{game}_event_id_count"] = stats.event_id.nunique()
        stats_features[f"world_{game}_session_id_count"] = stats.game_session.nunique()
        stats_features[f"world_{game}_n_actions"] = len(stats)
        result.update(stats_features)
        result.update({f"world_{game}_{k}": v for k, v in make_counters(stats, "event_id").items()})
        result.update({f"world_{game}_{k}": v for k, v in make_counters(stats, "event_code").items()})
    result.update(make_counters(history, "event_id"))
    result.update(make_counters(history, "event_code"))
    return result

In [6]:
train_features = process_installations(train_labels, train, process_log)

100%|██████████| 17690/17690 [1:24:37<00:00,  3.48it/s]


In [12]:
def get_duplicate_features(features):
    to_remove = []
    counter = 0
    feature_names=[f for f in features.columns if f not in ("installation_id", "game_session", "accuracy_group")]
    for feat_a in tqdm(feature_names):
        for feat_b in feature_names:
            if feat_a != feat_b and feat_a not in to_remove and feat_b not in to_remove:
                c = np.corrcoef(features[feat_a], features[feat_b])[0][1]
                if c > 0.995:
                    counter += 1
                    to_remove.append(feat_b)
                    #print('{}: FEAT_A: {} FEAT_B: {} - Correlation: {}'.format(counter, feat_a, feat_b, c))
    print(f"{len(to_remove)} features were removed ({round(len(to_remove)/len(feature_names)*100, 2)}% of all features)")
    return to_remove
    
    
duplicate_features = get_duplicate_features(train_features)

useful_features = [f for f in train_features.columns if f not in duplicate_features]
useful_features

100%|██████████| 2023/2023 [05:58<00:00,  5.65it/s]

1116 features were removed (55.17% of all features)


['title',
 'world',
 'n_activities',
 'n_games',
 'event_code_count',
 'event_id_count',
 'title_count',
 'session_id_count',
 'n_actions',
 'world_title_count',
 'n_correct_Scrub-A-Dub',
 'n_incorrect_Scrub-A-Dub',
 'global_ratio_Scrub-A-Dub',
 'n_correct_Scrub-A-Dub_0',
 'n_incorrect_Scrub-A-Dub 0',
 'ratio_Scrub-A-Dub_0',
 'n_correct_Scrub-A-Dub_1',
 'n_incorrect_Scrub-A-Dub_1',
 'ratio_Scrub-A-Dub_1',
 'Scrub-A-Dub_event_code_count',
 'Scrub-A-Dub_session_id_count',
 'Scrub-A-Dub_n_actions',
 'Scrub-A-Dub_4a09ace1',
 'Scrub-A-Dub_5a848010',
 'Scrub-A-Dub_5c3d2b2f',
 'Scrub-A-Dub_6d90d394',
 'Scrub-A-Dub_7040c096',
 'Scrub-A-Dub_c1cac9a2',
 'Scrub-A-Dub_cf82af56',
 'Scrub-A-Dub_f71c4741',
 'n_correct_All Star Sorting',
 'n_incorrect_All Star Sorting',
 'global_ratio_All Star Sorting',
 'n_correct_All Star Sorting_0',
 'n_incorrect_All Star Sorting 0',
 'ratio_All Star Sorting_0',
 'n_correct_All Star Sorting_1',
 'n_incorrect_All Star Sorting 1',
 'ratio_All Star Sorting_1',
 'All S

In [13]:
train_features=train_features[useful_features].copy()

In [14]:
def lgb_quad_kappa(preds, true):
    true = true.get_label()
    #preds = preds.reshape((4, -1)).argmax(axis=0)
    preds = np.rint(preds)
    preds = np.maximum(0, preds)
    preds = np.minimum(3, preds)
    return "quad_kappa", quad_kappa(true, preds), True
    
    
def train_baseline(x_train,y_train, params=None):
    x_train_all, x_val_all,y_train_all,y_val_all = train_test_split(
        x_train,y_train,
        test_size=0.15,
        random_state=2019,
    )
    train_set = lgb.Dataset(x_train_all, y_train_all)
    val_set = lgb.Dataset(x_val_all, y_val_all)

#     params = {
#         'learning_rate': 0.01,
#         'bagging_fraction': 0.9,
#         'feature_fraction': 0.9,
#         'num_leaves': 14,
#         'lambda_l1': 0.1,
#         'lambda_l2': 1,
#         'metric': 'multiclass',
#         'objective': 'multiclass',
#         'num_classes': 4,
#         'random_state': 2019
#     }

    return lgb.train(params, train_set, num_boost_round=10000, early_stopping_rounds=2000, valid_sets=[val_set], verbose_eval=100)#,
                    #feval=lgb_quad_kappa)

def make_features(df):
    return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values

def make_features_wrapper(train, test):
    def make_features(df):
        return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values
    
    return make_features(train), make_features(test) 


def make_predictions(model,x_test_all,y_test):
    preds=model.predict(x_test_all)
    #preds = np.rint(preds)
    #preds = np.maximum(0, preds)
    #preds = np.minimum(3, preds)
    return preds,y_test

In [16]:
def process_test_installations(test):
    test = test.sort_values("timestamp")
    test=test.groupby("installation_id").progress_apply(process_log).reset_index()
    test.columns = ["installation_id", "features"]
    result = []
    for i, installation_id, feature in test.itertuples():
        result.append(feature)
        feature["installation_id"]=installation_id
    return pd.DataFrame(result).fillna(-1)
test_features=process_test_installations(test)

for useful_feature in useful_features:
    if useful_feature not in test_features.columns:
        test_features[useful_feature]=-1
        print("Missing feature", useful_feature)

test_features=test_features[[c for c in useful_features if c != "accuracy_group"]].copy()

100%|██████████| 1000/1000 [03:30<00:00,  4.74it/s]


Missing feature accuracy_group
Missing feature world_Mushroom Sorter (Assessment)_eb2c19cd
Missing feature Bubble Bath_29a42aea
Missing feature Mushroom Sorter (Assessment)_13f56524
Missing feature world_Mushroom Sorter (Assessment)_13f56524
Missing feature Chest Sorter (Assessment)_bfc77bd6
Missing feature world_Chest Sorter (Assessment)_bfc77bd6
Missing feature world_Bubble Bath_29a42aea
Missing feature Dino Drink_ab4ec3a4
Missing feature world_Dino Drink_ab4ec3a4
Missing feature Happy Camel_0ce40006
Missing feature world_Cauldron Filler (Assessment)_9554a50b
Missing feature Dino Dive_119b5b02
Missing feature world_Dino Dive_119b5b02
Missing feature Cart Balancer (Assessment)_ecc6157f
Missing feature world_Cart Balancer (Assessment)_ecc6157f
Missing feature world_Bird Measurer (Assessment)_6077cc36
Missing feature Pan Balance_e4d32835
Missing feature world_Pan Balance_e4d32835
Missing feature Leaf Leader_01ca3a3c
Missing feature world_Leaf Leader_01ca3a3c
Missing feature world_Happy 

In [17]:
subtrain_installations=pd.Series(train_features.installation_id.unique()).sample(frac=1., random_state=2019)
subtrain_features=train_features[train_features.installation_id.isin(subtrain_installations.values)].copy()
def check_hyperparams(params):
    print(params)
    if "max_depth" in params:
        params["max_depth"] = int(params["max_depth"])
    if "num_leaves" in params:
        params["num_leaves"] = int(params["num_leaves"])

    train_baseline_with_params = partial(train_baseline, params=params)
    cv=InstallationFold(n_splits=3)
    predictions = cross_validate(subtrain_features, subtrain_features.accuracy_group, make_features_wrapper, train_baseline_with_params, make_predictions,
                                cv=cv)
    return {
        "loss": np.mean([mean_squared_error(true, pred) for pred, true in predictions]),
        "status": STATUS_OK,
        "params": params
    }


def tune(check_params, n_tries=25, n_learning_rate_tries=15, learning_rate=None, n_estimators=10_000):        
    if learning_rate is None:
        learning_rate_space = {
            'learning_rate': hp.loguniform("learning_rate", np.log(0.005), np.log(0.3)),
            'metric': 'rmse',
            'objective': 'rmse',
            #'num_classes': 4,
            'random_state': 2019,
            "n_estimators": n_estimators,

        }
        trials = Trials()
        result = fmin(check_params,
                      learning_rate_space, tpe.suggest, n_learning_rate_tries, trials=trials)
        print(result)
        learning_rate = round(trials.best_trial["result"]["params"]["learning_rate"], 3)

    param_space = {
        'metric': 'rmse',
        'objective': 'rmse',
        #'num_classes': 4,
        'lambda_l1': hp.uniform("lamba_l1", 1e-10, 1),
        'lambda_l2': hp.uniform("lambda_l2", 1e-10, 1),
        'random_state': 2019,
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": hp.quniform("max_depth", 2, 16, 1),
        "num_leaves": hp.choice("num_leaves", [3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047, 4095]),
        "subsample": hp.quniform("subsample", 0.01, 1, 0.01),
        "feature_fraction": hp.quniform("feature_fraction", 0.01, 1, 0.01),
    }

    trials = Trials()
    fmin(check_params,
         param_space, tpe.suggest, n_tries, trials=trials)
    best_params = trials.best_trial["result"]["params"]
    return best_params

In [19]:
best_params=tune(check_hyperparams, n_tries=25, n_learning_rate_tries=10)

{'learning_rate': 0.01406473596255922, 'metric': 'rmse', 'n_estimators': 10000, 'objective': 'rmse', 'random_state': 2019}
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.03395                       
[200]	valid_0's rmse: 1.00353                       
[300]	valid_0's rmse: 0.994464                      
[400]	valid_0's rmse: 0.993262                      
[500]	valid_0's rmse: 0.993024                      
[600]	valid_0's rmse: 0.993397                      
[700]	valid_0's rmse: 0.994321                      
[800]	valid_0's rmse: 0.995694                      
[900]	valid_0's rmse: 0.997675                      
[1000]	valid_0's rmse: 0.999413                     
[1100]	valid_0's rmse: 1.001                        
[1200]	valid_0's rmse: 1.00237                      
[1300]	valid_0's rmse: 1.00314                      
[1400]	valid_0's rmse: 1.00402                      
[1500]	valid_0's rmse: 1.00447                      
[1600]	valid_0's rmse: 1.00524                      
[1700]	valid_0's rmse: 1.00672                      
[1800]	valid_0's rmse: 1.0079       

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds               
[100]	valid_0's rmse: 1.00142                                                
[200]	valid_0's rmse: 1.02232                                                
[300]	valid_0's rmse: 1.03152                                                
[400]	valid_0's rmse: 1.03867                                                
[500]	valid_0's rmse: 1.04822                                                
[600]	valid_0's rmse: 1.05175                                                
[700]	valid_0's rmse: 1.05509                                                
[800]	valid_0's rmse: 1.05892                                                
[900]	valid_0's rmse: 1.06153                                                
[1000]	valid_0's rmse: 1.06426                                               
[1100]	valid_0's rmse: 1.06629                                               
[1200]	valid_0's rmse: 1.068                                    

...

Training until validation scores don't improve for 2000 rounds                 
[100]	valid_0's rmse: 1.08945                                                  
[200]	valid_0's rmse: 1.0365                                                   
[300]	valid_0's rmse: 1.01654                                                  
[400]	valid_0's rmse: 1.00611                                                  
[500]	valid_0's rmse: 1.00108                                                  
[600]	valid_0's rmse: 0.997933                                                 
[700]	valid_0's rmse: 0.996237                                                 
[800]	valid_0's rmse: 0.995029                                                 
[900]	valid_0's rmse: 0.994632                                                 
[1000]	valid_0's rmse: 0.994713                                                
[1100]	valid_0's rmse: 0.994415                                                
[1200]	valid_0's rmse: 0.994549         

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds                 
[100]	valid_0's rmse: 1.08932                                                  
[200]	valid_0's rmse: 1.03677                                                  
[300]	valid_0's rmse: 1.0159                                                   
[400]	valid_0's rmse: 1.00573                                                  
[500]	valid_0's rmse: 1.00072                                                  
[600]	valid_0's rmse: 0.998454                                                 
[700]	valid_0's rmse: 0.997125                                                 
[800]	valid_0's rmse: 0.995982                                                 
[900]	valid_0's rmse: 0.995563                                                 
[1000]	valid_0's rmse: 0.995406                                                
[1100]	valid_0's rmse: 0.994874                                                
[1200]	valid_0's rmse: 0.994621         

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds                 
[100]	valid_0's rmse: 1.07174                                                  
[200]	valid_0's rmse: 1.01982                                                  
[300]	valid_0's rmse: 1.00264                                                  
[400]	valid_0's rmse: 0.996705                                                 
[500]	valid_0's rmse: 0.994553                                                 
[600]	valid_0's rmse: 0.994182                                                 
[700]	valid_0's rmse: 0.993905                                                 
[800]	valid_0's rmse: 0.993853                                                 
[900]	valid_0's rmse: 0.994178                                                 
[1000]	valid_0's rmse: 0.99471                                                 
[1100]	valid_0's rmse: 0.995248                                                
[1200]	valid_0's rmse: 0.995476         

In [20]:
best_params

{'feature_fraction': 0.67,
 'lambda_l1': 0.9431992357093691,
 'lambda_l2': 0.2004387038895958,
 'learning_rate': 0.01,
 'max_depth': 13,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.68}

What was selected on 100% of the data

```
{'feature_fraction': 0.44,
 'lambda_l1': 0.2342366019673217,
 'lambda_l2': 0.20913848894987938,
 'learning_rate': 0.04,
 'max_depth': 9,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.86}
```

In [21]:
# best_params={'feature_fraction': 0.58,
#  'lambda_l1': 0.45619796864269707,
#  'lambda_l2': 0.033257384218246686,
#  'learning_rate': 0.007,
#  'max_depth': 14,
#  'metric': 'multiclass',
#  'n_estimators': 10000,
#  'num_classes': 4,
#  'num_leaves': 31,
#  'objective': 'multiclass',
#  'random_state': 2019,
#  'subsample': 0.9500000000000001}

In [22]:
with open("../dsb2019/models/regression_baseline_eventid_bag_params.json", "w") as f:
    json.dump(best_params, f)

In [23]:
baseline_model=train_baseline(train_features.drop(["installation_id", "accuracy_group"], axis=1), train_features.accuracy_group.values, 
               params=best_params)

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.06583
[200]	valid_0's rmse: 1.00614
[300]	valid_0's rmse: 0.987652
[400]	valid_0's rmse: 0.980901
[500]	valid_0's rmse: 0.977921
[600]	valid_0's rmse: 0.976571
[700]	valid_0's rmse: 0.976453
[800]	valid_0's rmse: 0.976687
[900]	valid_0's rmse: 0.976809
[1000]	valid_0's rmse: 0.977259
[1100]	valid_0's rmse: 0.977601
[1200]	valid_0's rmse: 0.977982
[1300]	valid_0's rmse: 0.977905
[1400]	valid_0's rmse: 0.97811
[1500]	valid_0's rmse: 0.978354
[1600]	valid_0's rmse: 0.979004
[1700]	valid_0's rmse: 0.979261
[1800]	valid_0's rmse: 0.979351
[1900]	valid_0's rmse: 0.979819
[2000]	valid_0's rmse: 0.98028
[2100]	valid_0's rmse: 0.98091
[2200]	valid_0's rmse: 0.981174
[2300]	valid_0's rmse: 0.981666
[2400]	valid_0's rmse: 0.982127
[2500]	valid_0's rmse: 0.982513
[2600]	valid_0's rmse: 0.982887
[2700]	valid_0's rmse: 0.983264
Early stopping, best iteration is:
[708]	valid_0's rmse: 0.97634


In [24]:
predictions = cross_validate(train_features, train_features.accuracy_group, make_features_wrapper, partial(train_baseline, params=best_params), 
                             make_predictions)
np.mean([mean_squared_error(true, pred) for pred, true in predictions]), [mean_squared_error(true, pred) for pred, true in predictions]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.07251
[200]	valid_0's rmse: 1.01616
[300]	valid_0's rmse: 0.997589
[400]	valid_0's rmse: 0.991165
[500]	valid_0's rmse: 0.988518
[600]	valid_0's rmse: 0.987586
[700]	valid_0's rmse: 0.987135
[800]	valid_0's rmse: 0.987081
[900]	valid_0's rmse: 0.986889
[1000]	valid_0's rmse: 0.987243
[1100]	valid_0's rmse: 0.987755
[1200]	valid_0's rmse: 0.988074
[1300]	valid_0's rmse: 0.988519
[1400]	valid_0's rmse: 0.98913
[1500]	valid_0's rmse: 0.989855
[1600]	valid_0's rmse: 0.990163
[1700]	valid_0's rmse: 0.990536
[1800]	valid_0's rmse: 0.991234
[1900]	valid_0's rmse: 0.992049
[2000]	valid_0's rmse: 0.992675
[2100]	valid_0's rmse: 0.993278
[2200]	valid_0's rmse: 0.994134
[2300]	valid_0's rmse: 0.994992
[2400]	valid_0's rmse: 0.9958
[2500]	valid_0's rmse: 0.996723
[2600]	valid_0's rmse: 0.997432
[2700]	valid_0's rmse: 0.99791
[2800]	valid_0's rmse: 0.998575
[2900]	valid_0's rmse: 0.999084
Early stopping, best ite

(0.9813635389251534,
 [0.9503865319415311,
  0.9794837771733715,
  0.943886415123229,
  0.9956933998414647,
  1.037367570546171])

In [25]:
baseline_model.save_model(str(MODELS_DIR / "regression_baseline__eventid_bag.lgb"))

In [26]:
features, target = make_features(train_features)
prediction=baseline_model.predict(features)
clf = ThresholdClassifier()
clf.fit(prediction, target)

100%|██████████| 1000/1000 [00:24<00:00, 40.22it/s, best loss: -0.7045007373455217]


ThresholdClassifier(n_iter=1000, random_state=2019)

In [27]:
clf.coef_

[1.0539541002386488, 1.6792994887656274, 2.247341408420393]